# ratings.ipynb

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_json('../data/ratings.json')
data

,Alex Roberts,Jillian Hobart,John Carson,Melissa Jones,Michael Henry,Michelle Peterson,William Reynolds
Anger Management,NaN,3.0,3.0,2,NaN,1.5,NaN
Fracture,5.0,4.0,3.5,3,4.0,5.0,3.5
Inception,3.0,NaN,2.5,3,NaN,3.0,2.5
Jerry Maguire,3.0,4.5,3.0,3,NaN,3.0,4.0
Pulp Fiction,4.0,3.5,3.5,4,4.5,3.5,3.0
Serendipity,3.5,2.5,2.5,2,1.0,3.5,NaN


In [4]:
simmat = data.corr()
simmat

,Alex Roberts,Jillian Hobart,John Carson,Melissa Jones,Michael Henry,Michelle Peterson,William Reynolds
Alex Roberts,1.000000,0.028571,0.747018,0.211289,0.662849,0.963796,0.134840
Jillian Hobart,0.028571,1.000000,0.566947,0.566947,0.893405,0.314970,1.000000
John Carson,0.747018,0.566947,1.000000,0.594089,0.991241,0.396059,0.404520
Melissa Jones,0.211289,0.566947,0.594089,1.000000,0.924473,0.411765,-0.258199
Michael Henry,0.662849,0.893405,0.991241,0.924473,1.000000,0.381246,-1.000000
Michelle Peterson,0.963796,0.314970,0.396059,0.411765,0.381246,1.000000,0.204598
William Reynolds,0.134840,1.000000,0.404520,-0.258199,-1.000000,0.204598,1.000000


In [26]:

# 基于相似度矩阵完成推荐业务
login_user = 'William Reynolds'
# 根据矩阵找到login_user的相似用户(去掉自己，大小排序, 去掉反相关用户)
simusers = simmat[login_user].sort_values(ascending=False)
simusers = simusers.drop(login_user)
simusers = simusers[simusers>0]
simusers

Jillian Hobart       1.000000
John Carson          0.404520
Michelle Peterson    0.204598
Alex Roberts         0.134840
Name: William Reynolds, dtype: float64

In [27]:
# 看一下相似用户喜欢的电影，哪些是login_user没看过的，按照推荐度排序返回即可。
# {movie: [[3,3,2,3], [0.9, 0.3, 0.2, 0.6]]}
reco_movies = {}
for simuser, simscore in simusers.items():
    # 找到simuser看过的 而 登录用户没有看过的电影，保存起来
    movies = data[simuser].dropna()
    for movie, score in movies.items():
        if pd.isna(data[login_user][movie]):  # 找到召回的电影
            if movie not in reco_movies:
                reco_movies[movie] = [[], []]
            reco_movies[movie][0].append(score)
            reco_movies[movie][1].append(simscore)
            
print(reco_movies)

{'Anger Management': [[3.0, 3.0, 1.5], [1.0, 0.40451991747794525, 0.20459830184114206]], 'Serendipity': [[2.5, 2.5, 3.5, 3.5], [1.0, 0.40451991747794525, 0.20459830184114206, 0.13483997249264842]]}


In [28]:
# 为推荐字典按照value的均值进行排序
sorted_movie_list = \
    sorted(reco_movies.items(), 
           key=lambda x: np.average(x[1][0], weights=x[1][1]), reverse=True)
sorted_movie_list

[('Anger Management',
  [[3.0, 3.0, 1.5], [1.0, 0.40451991747794525, 0.20459830184114206]]),
 ('Serendipity',
  [[2.5, 2.5, 3.5, 3.5],
   [1.0, 0.40451991747794525, 0.20459830184114206, 0.13483997249264842]])]